In [ ]:
import pandas as pd
dates = pd.read_csv('data\MovLen_Data\dates.csv', sep=',')
mov = pd.read_csv('data\MovLen_Data\movies.csv', sep=',')
r1 = pd.read_csv('data\MovLen_Data\ings1.csv', sep=',')
r2 = pd.read_csv('data\MovLen_Data\ings2.csv', sep=',')
ratings = pd.concat([r1, r2], ignore_index=True) # Объединение таблиц в одну
ratings = ratings.drop_duplicates(ignore_index=True) # Удаление дубликатов
ratings_dates = pd.concat([ratings, dates], axis=1) # Объединение таблиц 
ratings_dates

In [ ]:
import pandas as pd
import os

def concat_user_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data],axis=0, ignore_index=True)
        data = data.drop_duplicates()
    return data


In [ ]:
joined_left = ratings_dates.join(
    mov, # Название второй таблицы
    rsuffix='_right', # 
    how='left' # Присоединение первичной таблицы ко вторичной
)
joined_left

In [ ]:
joined = ratings_dates.join(
    mov.set_index('movieId'), #
    on='movieId', #
    how='left' #
)
joined

In [ ]:
merged = ratings_dates.merge(
    mov,
    on='movieId',
    how='left'
)
merged.head()

In [ ]:
merged2 = ratings_dates.merge(
    mov,
    on='movieId',
    how='outer'
)
merged2.tail()

In [ ]:
merge_ratings = r1.merge(r2, how='outer')
merge_ratings

In [ ]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})


purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged_list = items_df.merge(purchase_df, how='inner', on='item_id')
income = (merged_list['price'] * merged_list['stock_count']).sum()
income

In [ ]:
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None


rm = pd.read_csv('data\MovLen_Data\RM.csv', sep=',')
rc = rm.copy()
joined['year_released'] = joined['title'].apply(get_year_release)
joined.info('year_released')

In [ ]:
mask = joined['year_released'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()

In [ ]:
mask = joined['year_released'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()

In [ ]:
joined.groupby('userId')['genres'].nunique().sort_values(ascending=False)

In [ ]:
joined.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by='mean', ascending=False)

In [ ]:
mask = joined['year_released'] == 2018
joined[mask].groupby('genres')['rating'].agg(['mean', 'count', 'max']).sort_values(by='count', ascending=False)

In [ ]:
joined['date'] = pd.to_datetime(joined['date'], dayfirst=True)
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(index='year_rating',
                       columns='genres',
                       values='rating',
                       aggfunc='mean')
pivot

In [50]:
ords = pd.read_csv('data\orders.csv', sep=';')
prods = pd.read_csv('data\products.csv', sep=';')
prods

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [64]:
order_products = ords.merge(prods,
                            left_on='ID товара',
                            right_on='Product_ID',
                            how='left')
order_products
order_products['profit'] = (order_products['Количество'] * order_products['Price'])
order_products[order_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['profit'].sum().sort_values()

ID Покупателя
1        0.0
8     1344.0
5    13043.0
7    17096.0
Name: profit, dtype: float64